# 数据载入

In [1]:
import pandas as pd
import os
from collections import Counter
import numpy as np

import torch

In [2]:
pd.set_option('display.max_colwidth', -1)

C:\Users\c'y'y\AppData\Local\Temp\ipykernel_8392\3652881321.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [3]:
poi_path =r'E:\xpj\research\POI\psqlcsv\rio.csv'
poi_prompt_path =r'E:\xpj\research\POI\result\rio_poi_prompt.xlsx'
image_path = r'E:\xpj\research\POI\psqlcsv\rio_img_location.csv'
image_poi_path = r'E:\xpj\research\POI\psqlcsv\rio_img_poi.csv'
desc_path =r'E:\xpj\research\POI\desc_merge\rio_desc_merged.csv'
best_poi_path = r'E:\xpj\research\POI\result\rio_best_poi.xlsx'
desc_poi_path = r'E:\xpj\research\POI\result\rio_desc_poi.xlsx'

## poi数据（全面）

In [4]:
poi_data = pd.read_csv(poi_path,encoding='utf-8')
# poi_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_poi.csv",encoding='ANSI')
poi_df = pd.DataFrame(poi_data)

In [5]:
poi_df

,id,geom,osm_id,name,barrier,highway,ref,address,is_in,place,man_made,other_tags
0,1,0101000020E61000008978EBFCDB5B47C0E2D7FE84698E37C0,577242,NaN,NaN,speed_camera,NaN,NaN,NaN,NaN,NaN,"""maxspeed""=>""90"",""maxspeed:hgv""=>""60"""
1,2,0101000020E61000000B2CDBE2754E47C0388F79D3E38537C0,597434,NaN,NaN,motorway_junction,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0101000020E61000004599B278569B45C0E28F47156AFC36C0,25038601,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0101000020E610000073BD6DA6429B45C07EA026440AFB36C0,25038605,NaN,NaN,crossing,NaN,NaN,NaN,NaN,NaN,"""traffic_calming""=>""table"""
4,5,0101000020E6100000CFFEF62B535B47C0DF0A7261EE8E37C0,25619567,NaN,NaN,motorway_junction,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
106079,106080,0101000020E6100000F1811DFF058E45C0792B01E77BE836C0,11382497947,O Galeto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""amenity""=>""restaurant"""
106080,106081,0101000020E6100000C652245F098E45C05BADB8DD70E836C0,11382509462,Drogarias Tamoio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""amenity""=>""pharmacy"",""healthcare""=>""pharmacy"""
106081,106082,0101000020E6100000A9007388138E45C057C62BB583E836C0,11382516664,Velocity/kore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""leisure""=>""fitness_centre"",""sport""=>""fitness"""
106082,106083,0101000020E61000009F97E546EC8D45C0C3FEA14F9AE836C0,11382530023,Princesa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""amenity""=>""marketplace"",""building""=>""yes"",""opening_hours""=>""07:30-21:00"""


In [6]:
poi_df.columns

Index(['id', 'geom', 'osm_id', 'name', 'barrier', 'highway', 'ref', 'address',
       'is_in', 'place', 'man_made', 'other_tags'],
      dtype='object')

 # 下面第一块可以不做

In [7]:
# save tag list to txt

# poi_df_nec=poi_df.loc[:,['id', 'name', 'barrier', 'highway','ref', 'address', 'place', 'man_made', 'other_tags']]
# poi_df_ot =poi_df_nec.loc[:,['other_tags']]
# tag_list = []
# def collect_tags(x):
#     # print(x['other_tags'])
    
#     if  pd.notna(x['other_tags']):
#         # print(x['other_tags'])
#         tags = x['other_tags'].split("\",\"")
#         # print(tags)
#         for tag in tags:
#             try:
#                 tag = tag.replace("\"","")
#                 t = tag.split("=>")[0]
#                 if t not in tag_list:
#                     tag_list.append(t)
#             except:
#                 pass
#     # return tag_list

# result = poi_df_ot.apply(lambda x: collect_tags(x), axis=1, result_type='expand')
# # result
# tag_list.sort()
# tag_list
# # 保存tag_list到txt
# with open(r'E:\xpj\research\POI\chennai_tag_list.txt', 'w') as f:
#     for item in tag_list:
#         f.write("%s\n" % item)

# poi_df_ot

# def genSent(x):
#     tags = None
#     b = True    
#     # sen = "The imformation of this point contains that "
#     sen = "Nearby,"
#     for key in ['name','barrier', 'highway','address','place','man_made']:
#         if not x[key].isna().iloc[0]:
#             # print(x[key])
#             sen = sen + key +" is " + x[key].iloc[0] + ","
#     if not x['other_tags'].isna().iloc[0]:
#         tags = x['other_tags'].iloc[0].split("\",\"")
#         for tag in tags:
#             try:
#                 tag = tag.replace("\"","")
#                 t = tag.split("=>")[0]
#                 k = tag.split("=>")[1]
#                 if "name:" in t and t != "name:en":
#                     pass
#                 sen = sen + t +" is " + k + ","
#             except:
#                 pass

#     sen = sen + "located in " + x['NAME_2'].iloc[0]
    
#     # return sen,b
    
#     return sen
# # poi_df_nec


## 照片信息

In [7]:
pic_data = pd.read_csv(image_path,encoding='utf-8')
# pic_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_points.csv",encoding='ANSI')
pic_df = pd.DataFrame(pic_data)

In [8]:
pic_df

,id,geom,captured_a,camera_mak,created_at,sequences_,pano,user_key,username,img_key,ca,longitude,latitude,img_id,lon,lat,layer,path
0,47099,0101000020E61000003F7A79F0C9AE45C0E9FF6673E6FE36C0,2015/05/18 11:23:03.717,QCOM-AA,2015/05/18 14:52:19.580,crd3q2hfjyvknrgzx1hq4s,0,QnZSIBLP-0fNN3d-7v-Nsg,nighto,O8cJXNVujCuc7lodVZxjIQ,278.3590,-43.365538,-22.995704,NaN,NaN,NaN,rio_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/rio_img_location.shp
1,47100,0101000020E61000003460353BF7B445C0448E08217BFC36C0,2016/07/10 13:42:49.589,none,2016/07/29 14:29:52.281,efAYI6ZY6nOcElKV2rlozQ,0,QnZSIBLP-0fNN3d-7v-Nsg,nighto,FfAPd-qnWjlAennAnYz54g,34.7201,-43.413795,-22.986254,NaN,NaN,NaN,rio_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/rio_img_location.shp
2,47101,0101000020E61000003F7A79F0C9AE45C0E9FF6673E6FE36C0,2015/05/18 11:23:03.717,QCOM-AA,2015/05/18 14:52:19.580,crd3q2hfjyvknrgzx1hq4s,0,QnZSIBLP-0fNN3d-7v-Nsg,nighto,7G7UMIUMXAG9RjZ71hPHBw,277.8210,-43.365538,-22.995704,NaN,NaN,NaN,rio_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/rio_img_location.shp
3,47102,0101000020E6100000FF942A51F6B445C00AC446B478FC36C0,2016/07/10 13:42:49.589,none,2016/07/29 14:29:52.281,efAYI6ZY6nOcElKV2rlozQ,0,QnZSIBLP-0fNN3d-7v-Nsg,nighto,FbA-NO48-w0xViG8KH7vaw,34.7201,-43.413767,-22.986217,NaN,NaN,NaN,rio_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/rio_img_location.shp
4,47103,0101000020E61000003F7A79F0C9AE45C0E9FF6673E6FE36C0,2015/05/18 11:23:03.717,QCOM-AA,2015/05/18 14:52:19.580,crd3q2hfjyvknrgzx1hq4s,0,QnZSIBLP-0fNN3d-7v-Nsg,nighto,WMse9Wt0r_F4UevP5zrXwA,275.6530,-43.365538,-22.995704,NaN,NaN,NaN,rio_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/rio_img_location.shp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161574,47094,0101000020E61000003F027FF8F9B445C01D6CFB6882FC36C0,2016/07/10 13:42:49.589,none,2016/07/29 14:29:52.281,efAYI6ZY6nOcElKV2rlozQ,0,QnZSIBLP-0fNN3d-7v-Nsg,nighto,I4-lVU7TOf7uXygpwTNOYg,34.7201,-43.413879,-22.986365,NaN,NaN,NaN,rio_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/rio_img_location.shp
161575,47095,0101000020E61000003F7A79F0C9AE45C0E9FF6673E6FE36C0,2015/05/18 11:23:03.717,QCOM-AA,2015/05/18 14:52:19.580,crd3q2hfjyvknrgzx1hq4s,0,QnZSIBLP-0fNN3d-7v-Nsg,nighto,MiH-rASORt5ENI7kew1N7Q,273.7280,-43.365538,-22.995704,NaN,NaN,NaN,rio_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/rio_img_location.shp
161576,47096,0101000020E61000000A37740EF9B445C0E2A139FC7FFC36C0,2016/07/10 13:42:49.589,none,2016/07/29 14:29:52.281,efAYI6ZY6nOcElKV2rlozQ,0,QnZSIBLP-0fNN3d-7v-Nsg,nighto,QWvPx9VJR37FVVEaRNuZ2w,34.7201,-43.413851,-22.986328,NaN,NaN,NaN,rio_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/rio_img_location.shp
161577,47097,0101000020E61000003F7A79F0C9AE45C0E9FF6673E6FE36C0,2015/05/18 11:23:03.717,QCOM-AA,2015/05/18 14:52:19.580,crd3q2hfjyvknrgzx1hq4s,0,QnZSIBLP-0fNN3d-7v-Nsg,nighto,bBePH1PmCAp5m9_QyxM43A,279.5140,-43.365538,-22.995704,NaN,NaN,NaN,rio_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/rio_img_location.shp


## 照片poi关联

In [10]:
pic_poi_data = pd.read_csv(image_poi_path,encoding='utf-8')
# pic_poi_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_pois_to_points.csv",encoding='ANSI')
pic_poi_df = pd.DataFrame(pic_poi_data)

In [13]:
pic_poi_df

,img_key,id1,distance1,id2,distance2,id3,distance3,id4,distance4,id5,distance5
0,002s3LqUHs-4gWRcYaiBkw,34793,17.721983,34781.0,22.808563,34593.0,24.099713,34581.0,37.013036,34689.0,38.000283
1,003XithOyLilNhuQGbaEZg,40753,0.377198,40752.0,5.468711,40754.0,8.865106,40755.0,14.859372,67606.0,18.802007
2,004aS2auM9uZDCfs6No_aQ,34865,12.967810,34603.0,24.182185,34820.0,26.507287,34694.0,31.367598,34869.0,31.629926
3,004EJBeTzGPJ2xf4dnkKjA,4014,106.115749,3331.0,109.034853,19499.0,111.012940,3334.0,114.621619,2727.0,116.903414
4,005N-4kg7UzNLGCWC5P1Mg,41183,39.366317,44733.0,75.702342,41181.0,156.647688,24351.0,245.942360,39842.0,258.698243
...,...,...,...,...,...,...,...,...,...,...,...
64518,ZZxIjo2NiKAPjqwceHD1zw,29519,63.996659,42002.0,75.143907,57849.0,75.758164,5927.0,76.237239,57848.0,76.542434
64519,ZzXRUvKOW24VAHD95eh1yQ,21267,14.097283,21225.0,26.195435,58935.0,27.033246,21226.0,27.244517,21229.0,29.253731
64520,ZzYYYg6qM0_xcVcF5372YQ,55410,28.429007,67388.0,29.996281,52567.0,58.866504,56176.0,61.577739,39586.0,83.735081
64521,zzZ8UbiqxCBY-XBpTWu0Ng,18620,62.501259,18617.0,86.176798,59543.0,87.672036,43253.0,99.532265,43254.0,100.608176


## 照片描述

In [11]:
desc_pd =pd.read_csv(desc_path,encoding='utf-8')
desc_pd

,key,prompt
0,_Hl_r3FRhTeYWkNOSoC9jw,"arafed view of a fenced off area with a street sign, a matte painting inspired by Washington Allston, featured on shutterstock, ashcan school, small fence, fence line, white block fence"
1,_hzARtgodzk_0iEkMsHfWg,"arafed view of a paved road with a fence and grass, a matte painting by Tadao Ando, polycount, ecological art, grassy hill, landslide road, grass and weeds”"
2,_JrLfcgray_wVFbNp6DqgA,"a view of a street with a telephone pole and bushes, a photo inspired by Eduardo Lefebvre Scovell, pixiv, ecological art, overgrown vegetation, found on google street view, overgrown with vegetation"
3,_K1nHlsKBSLNqBuGK8P-0A,"arafed house with a fence and a gate in front of it, a detailed matte painting inspired by Juan Villafuerte, featured on shutterstock, barbizon school, footbridges between houses, in front of a two story house, inside a gang hideout"
4,_K7FOnA4OpLuBCQiafVOCg,"graffiti on a wall in an alley with a building in the background, graffiti art by Derf, behance, funk art, grafity, graffiti art style, graffiti style"
...,...,...
747564,ynJo3obus0JkZOgC5dX9YA,"cars are driving down the road in front of tall buildings, a matte painting inspired by Victor Meirelles, pexels, new objectivity, anamorphic lens flare, optical lens flare fx, anamorphic wide angle lens"
747565,9z7pOSMyF5rh0ZHM6bS6Yw,"arafed bus driving down a street with cars on it, a matte painting inspired by Nadir Afonso, tumblr, stuckism, dashcam footage, panoramic photography, anamorphic wide angle lens"
747566,Pz_PN-KMXaDd81bBDSFlvQ,"cars are driving down the street in front of a bridge, a photo by Osvaldo Romberg, tumblr, excessivism, avenida paulista, traffic signs, são paulo"
747567,O9Vk9GshlT7iTL0D0a1vJA,"cars driving down a highway with tall buildings in the background, a matte painting by Jacob Willemszoon de Wet, tumblr, precisionism, dashcam footage, found on google street view, google maps street view"


# 数据处理

## poi数据预处理

In [12]:
poi_df_nec=poi_df.loc[:,['id', 'name', 'barrier', 'highway', 'address', 'place', 'man_made', 'other_tags']]

In [13]:
poi_df_nec

,id,name,barrier,highway,address,place,man_made,other_tags
0,1,NaN,NaN,speed_camera,NaN,NaN,NaN,"""maxspeed""=>""90"",""maxspeed:hgv""=>""60"""
1,2,NaN,NaN,motorway_junction,NaN,NaN,NaN,NaN
2,3,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN
3,4,NaN,NaN,crossing,NaN,NaN,NaN,"""traffic_calming""=>""table"""
4,5,NaN,NaN,motorway_junction,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
106079,106080,O Galeto,NaN,NaN,NaN,NaN,NaN,"""amenity""=>""restaurant"""
106080,106081,Drogarias Tamoio,NaN,NaN,NaN,NaN,NaN,"""amenity""=>""pharmacy"",""healthcare""=>""pharmacy"""
106081,106082,Velocity/kore,NaN,NaN,NaN,NaN,NaN,"""leisure""=>""fitness_centre"",""sport""=>""fitness"""
106082,106083,Princesa,NaN,NaN,NaN,NaN,NaN,"""amenity""=>""marketplace"",""building""=>""yes"",""opening_hours""=>""07:30-21:00"""


In [ ]:
# poi_df_nec.to_csv(r"E:\xpj\research\POI\poi_df_nec.csv")

# 硬提示计算相似度，找最佳poi

In [14]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# 加载BERT模型和分词器
tokenizer = AutoTokenizer.from_pretrained(r"E:\xpj\models\bert-base-uncased")
model = AutoModel.from_pretrained(r"E:\xpj\models\bert-base-uncased")

In [15]:
# 定义计算相似度的函数
def calc_similarity(s1, s2):
    # 对句子进行分词，并添加特殊标记
    inputs = tokenizer([s1, s2], return_tensors='pt', padding=True, truncation=True)
    # print(inputs)

    # 将输入传递给BERT模型，并获取输出
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

    # 计算余弦相似度，并返回结果
    sim = np.dot(embeddings[0], embeddings[1]) / (np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1]))
    return sim


In [16]:

def isEnglish(s):
    if isinstance(s, str):
        return s.isascii()
    return False



# 从txt中读取tag_list
tag_ignore_list = []
with open(r'E:\xpj\research\POI\tag_ignore_list.txt', 'r') as f:
    for line in f.readlines():
        tag_ignore_list.append(line.strip())
# print(tag_ignore_list)

def genSent(x):
    tags = None
    b = True    
    # sen = "The imformation of this point contains that "
    sen = "Nearby,"
    for key in ['name','barrier', 'highway','address','place','man_made']:
        if isinstance(x[key],str) and not isEnglish(x[key]):
            continue
        # # 北京保留中文
        # if key == 'name' and not isEnglish(x['name'].iloc[0]):
        #     continue
        if not x[key].isna().iloc[0]:
            # print(x[key])
            sen = sen + key +" is " + x[key].iloc[0] + ","
    # print(sen)
    if not x['other_tags'].isna().iloc[0]:
        tags = x['other_tags'].iloc[0].split("\",\"")
        for tag in tags:
            try:
                tag = tag.replace("\"","")
                t = tag.split("=>")[0]
                k = tag.split("=>")[1]
                # 北京保留中文
                if not isEnglish(k) or not isEnglish(t):
                    print(k,t)
                    continue
                if list(t)[-1] == "=":
                    continue
                if t in tag_ignore_list:
                    continue
                # if "name:" in t and t != "name:en":
                #     pass
                # pass 的作用是什么也不做 跳过，continue的作用是执行下次for循环
                # print(t,k)
                sen = sen + t +" is " + k + ","
            except:
                pass

    # sen = sen + "located in " + x['NAME_2'].iloc[0]
    
    # return sen,b
    
    return sen

In [ ]:
# a="脿陇陇脿陇戮脿陇聹脿陇庐脿陇鹿脿陇戮脿陇脿陇陋脿楼聟脿陇虏脿楼聡脿陇脿陇鹿脿楼聣脿陇聼脿楼聡脿陇"
# # print(isEnglish(a))
# b="à¤­à¤¾à¤°à¤¤à¥à¤à¤¹à¤µà¤¾à¤®à¤¾à¤¨ à¤µà¤¿à¤­à¤¾à¤"
# print(isEnglish(b))
# print(b.encode('utf-8').decode('ISO-8859-1'))
# s = "啶ぞ啶班い啷€啶啶灌さ啶距ぎ啶距え 啶掂た啶ぞ啶"
# print(s.encode('ansi').decode('iscii'))

将poi对应的自然语言提示存储到文件中(poi_nec最后插入一列提示)

In [17]:
def store_poi(x):
    # print(x)
    poi = x[0]
    # print(poi)    
    poi_im = poi_df_nec.loc[poi_df_nec['id']==poi]
    sen = genSent(poi_im)
    return sen
    # print(sen)


    

In [18]:
from tqdm import tqdm
result =[]
for index, row in tqdm(poi_df_nec.iterrows(), total=poi_df_nec.shape[0], desc="Processing"):
    # if index == 130:
    #     print(store_poi(row))
    result.append(store_poi(row))
# result = poi_df_nec.apply(lambda x: store_poi(x), axis=1, result_type='expand')
# poi_df_nec['poi_prompt']= result
# poi_df_nec['poi_prompt'] = result

Processing:   0%|          | 132/106084 [00:00<02:38, 667.61it/s]

Última Saída description
Última Saída description
Última Saída description
Última Saída exit_to
Autopista Régis operator
Autopista Régis operator
Бразилиа name:ab
ብ�� name:am
Głowa Cukru name:pl
Pão de Açúcar name:pt
Сахарная голова (Пан-ди-Асукар) name:ru
Rio-de-Ĵanejro alt_name:eo
PalãÚnga name:oym
Kulumuli ɨtu name:oym


Processing:   0%|          | 280/106084 [00:00<02:28, 711.91it/s]

Borne de trijonction: Brésil - France - Suriname name:fr
Rua São Salvador addr:street
Duque de Caxias
Petrópolis inscription
Pedra da Gávea name:de
Pedra da Gávea name:fr
Педра-да-Гавеа name:ru
Irmão Menor do Leblon alt_name
pt:Morro da Babilônia wikipedia


Processing:   0%|          | 505/106084 [00:00<02:23, 735.48it/s]

...

Niterói addr:city
Avenida Irene Lopes Sodré addr:street
Rua Vereador José Vicente Sobrinho addr:street
Rua Vereador José Vicente Sobrinho addr:street
Niterói addr:city
São Gonçalo addr:city
seg á sab 07:00-19:00 opening_hours
São Gonçalo addr:city
08 ás 17 opening_hours
José e Jhonatan operator
Niterói addr:city
Seg á sab 07 ás 18 opening_hours
Sérgio operator
Rua Domingues de Sá addr:street
Avenida Doutor Eugênio Borges addr:street


Processing: 100%|█████████▉| 105642/106084 [02:20<00:00, 751.34it/s]

Maricá addr:city
São Gonçalo addr:city
Parque Ecológico alt_name
Conjunto Nova Maré alt_name
Estrada de Jacarepaguá addr:street
Estrada de Jacarepaguá addr:street
Estrada de Jacarepaguá addr:street
Rua Maestro Felício Toledo addr:street
Rua Visconde de Pirajá addr:street


Processing: 100%|██████████| 106084/106084 [02:21<00:00, 749.83it/s]

São Gonçalo addr:city
São Gonçalo addr:city
São Gonçalo addr:city
Suécia addr:suburb
A Multisun é uma empresa líder e autoritaria em energia solar em Teresópolis, Brasil, e description
Rua Elói Teixeira addr:street
Universidade Estácio de Sá operator
Rua José Júlio da Cunha Raposo addr:street
Rua Vinícius de Moraes addr:street


In [21]:
import pandas as pd
print(pd.options)

In [22]:
# poi_df_nec.to_csv(r"E:\xpj\research\POI\poi_df_nec3.csv")
poi_df_nec['poi_prompt'] = result
poi_df_nec.to_excel(poi_prompt_path)
# poi_df_nec.to_csv(poi_prompt_path)


In [23]:
poi_df_nec =pd.read_excel(poi_prompt_path)
poi_df_nec

,Unnamed: 0,id,name,barrier,highway,address,place,man_made,other_tags,poi_prompt
0,0,1,NaN,NaN,speed_camera,NaN,NaN,NaN,"""maxspeed""=>""90"",""maxspeed:hgv""=>""60""","Nearby,highway is speed_camera,maxspeed is 90,maxspeed:hgv is 60,"
1,1,2,NaN,NaN,motorway_junction,NaN,NaN,NaN,NaN,"Nearby,highway is motorway_junction,"
2,2,3,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,"Nearby,highway is traffic_signals,"
3,3,4,NaN,NaN,crossing,NaN,NaN,NaN,"""traffic_calming""=>""table""","Nearby,highway is crossing,traffic_calming is table,"
4,4,5,NaN,NaN,motorway_junction,NaN,NaN,NaN,NaN,"Nearby,highway is motorway_junction,"
...,...,...,...,...,...,...,...,...,...,...
106079,106079,106080,O Galeto,NaN,NaN,NaN,NaN,NaN,"""amenity""=>""restaurant""","Nearby,name is O Galeto,amenity is restaurant,"
106080,106080,106081,Drogarias Tamoio,NaN,NaN,NaN,NaN,NaN,"""amenity""=>""pharmacy"",""healthcare""=>""pharmacy""","Nearby,name is Drogarias Tamoio,amenity is pharmacy,healthcare is pharmacy,"
106081,106081,106082,Velocity/kore,NaN,NaN,NaN,NaN,NaN,"""leisure""=>""fitness_centre"",""sport""=>""fitness""","Nearby,name is Velocity/kore,leisure is fitness_centre,sport is fitness,"
106082,106082,106083,Princesa,NaN,NaN,NaN,NaN,NaN,"""amenity""=>""marketplace"",""building""=>""yes"",""opening_hours""=>""07:30-21:00""","Nearby,name is Princesa,amenity is marketplace,building is yes,"


In [24]:
def find_best_poi(x):
    # print(x)
    img_key = x['img_key']
    # img_file = img_key+'.jpg'
    img_file =img_key
    try:
        img_desc = desc_pd.loc[desc_pd['key']==img_file]['prompt'].iloc[0]
    except:
        print("The description of picture " + img_file +" doesn't exist")
        return  None,None
    pois = x[1::2]
    pois_dis =x[2::2]
    # print(pois)
    similarities = []
    sentences = []
    for index,poi in enumerate(pois):
        # 设置距离阈值
        if pois_dis[index] is not None and pois_dis[index]<100:

            # # 根据id从poi_df_nec中找到poi_prompt
            try:
                sen = poi_df_nec.loc[poi_df_nec['id']==poi]['poi_prompt'].iloc[0]
            except Exception as e:
                print(e)
                print("The poi_prompt of poi " + poi +" doesn't exist")
                continue
            
            # poi_im = poi_df_nec.loc[poi_df_nec['id']==poi]
            # sen = genSent(poi_im)
            # try:
            # # 检查 DataFrame 是否为空
            #     sen = poi_df_nec.loc[poi_df_nec['id']==poi]['poi_prompt'].iloc[0]
            #     # 在此处理 sen 的其他操作

            # except Exception as e:
            #     print(e)
            #     # print("An error occurred while processing poi " + poi)
            #     continue

        
            similarity = calc_similarity(img_desc,sen)
            sentences.append(sen)
            similarities.append(similarity)
        
    if len(similarities) == 0:
        best_poi=None
        return best_poi,0
    elif similarities and max(similarities)>0.5:
        best_poi = pois[similarities.index(max(similarities))]
        return best_poi,max(similarities)

In [25]:
result = []
for _, row in tqdm(pic_poi_df.iterrows(), total=pic_poi_df.shape[0], desc="Processing"):
    result.append(find_best_poi(row))

# pic_poi_df[['best_poi', 'best_poi_score']] = result

Processing:   0%|          | 0/161152 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Processing:   0%|          | 426/161152 [02:25<12:04:05,  3.70it/s]

The description of picture 0a9UHflgJHdVNHHI6KvmZw doesn't exist


Processing:   1%|          | 1303/161152 [08:23<15:59:26,  2.78it/s]

The description of picture 0KALF3iCxVI-S12B0LRLOw doesn't exist


...

Processing: 100%|█████████▉| 160605/161152 [19:09:45<02:26,  3.73it/s]

The description of picture Zwsvbw4tJYwa409yC4tx7w doesn't exist


Processing: 100%|█████████▉| 160782/161152 [19:11:02<01:47,  3.44it/s]

The description of picture zxSEit_FoWY6qpyT5rTLzw doesn't exist


Processing: 100%|██████████| 161152/161152 [19:13:47<00:00,  2.33it/s]


In [26]:
result

[(44108.0, 0.821597),
 (95019.0, 0.72882783),
 (259, 0.6873395),
 (None, 0),
 (17517.0, 0.8029823),
 (83997, 0.7973194),
 (129.0, 0.74221754),
 (1825.0, 0.77338564),
 (36062.0, 0.7839863),
 (45066.0, 0.8145729),
 (None, 0),
 (1307.0, 0.8139825),
 (None, 0),
 (10680.0, 0.7356),
 (5950, 0.721549),
 (77015, 0.74153894),
 (None, 0),
 (49949.0, 0.8144732),
 (7962, 0.8045541),
 (55998.0, 0.74895906),
 (1797, 0.8015478),
 (46016, 0.8249052),
 (None, 0),
 (30392, 0.8192541),
 (56393.0, 0.7984466),
 (31570, 0.8034717),
 (48490.0, 0.8397093),
 (22454.0, 0.76573247),
 (94855, 0.7932124),
 (83739.0, 0.7819436),
 (101809.0, 0.77953774),
 (None, 0),
 (None, 0),
 (86726.0, 0.8040537),
 (3552, 0.8219561),
 (None, 0),
 (10918.0, 0.8256973),
 (860, 0.75199145),
 (35304, 0.743155),
 (1888.0, 0.7657286),
 (14860, 0.7501098),
 (88673.0, 0.72649854),
 (46028.0, 0.6886832),
 (94764.0, 0.7788339),
 (96605.0, 0.775698),
 (46908, 0.7533777),
 (26629.0, 0.7756727),
 (None, 0),
 (74085, 0.7266281),
 (88652, 0.811

In [27]:
result
result_df = pd.DataFrame(result,columns=['best_poi','best_poi_score']) # 将列表转换成DataFrame
pic_poi_df = pic_poi_df.join(result_df) # 将新的DataFrame添加到原始DataFrame中
# pic_poi_df.shape[0]

In [ ]:
# 上一块做了，这一块就不做了
#result = pic_poi_df.apply(lambda x: find_best_poi(x), axis=1,result_type='expand')

In [ ]:
#pic_poi_df[['best_poi','similarity']]= result

In [28]:
pic_poi_df

,img_key,id1,distance1,id2,distance2,id3,distance3,id4,distance4,id5,distance5,best_poi,best_poi_score
0,001Laaog8zdwYugH_HWEgw,21752,13.059461,21750.0,19.824303,21751.0,21.686056,44108.0,24.086535,21753.0,28.384339,44108.0,0.821597
1,001Q3_sTWsNcUd0yl-nJLA,94058,17.821758,42113.0,21.367698,95019.0,21.512402,42137.0,21.694559,90529.0,22.674062,95019.0,0.728828
2,0025pq4iX5JFSzLE4mzwKQ,259,98.612218,48095.0,116.037520,52036.0,142.219760,48096.0,148.991137,52502.0,152.942388,259.0,0.687339
3,002QPn5NUqaQIqgAXwrVlg,95628,125.756467,66004.0,167.137533,82889.0,177.383359,7174.0,196.736422,95632.0,228.807795,NaN,0.000000
4,002U0X8H3fkVK0dCi3ZRQA,93806,26.203495,93404.0,32.968048,17517.0,36.808813,22492.0,36.974101,93403.0,38.511392,17517.0,0.802982
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161147,ZzZ0-G17tbu8HO4rUrh4-A,2414,42.792871,71755.0,90.276320,71756.0,163.123758,66588.0,171.256074,66287.0,176.913615,2414.0,0.734527
161148,ZzZ9cMtim4UIwvi1I8vOAA,90160,51.994656,90161.0,70.898454,85677.0,102.030496,74035.0,136.287096,49739.0,140.106326,90160.0,0.794737
161149,_zzZDJ5eTaqBpSWRrJlqmA,59005,6.677372,94904.0,6.997275,59012.0,9.174672,59006.0,10.105857,59011.0,13.258052,94904.0,0.789188
161150,zZznk-22KYhH72kTEaqLjw,41881,36.733770,9531.0,46.433957,9549.0,58.756743,9548.0,59.055031,9554.0,76.311431,9531.0,0.813847


In [29]:
pic_poi_df.to_excel(best_poi_path)

In [30]:
df1 = pd.merge(pic_poi_df, poi_df_nec, left_on="best_poi",right_on = 'id',how="left")

In [31]:
df1

,img_key,id1,distance1,id2,distance2,id3,distance3,id4,distance4,id5,...,Unnamed: 0,id,name,barrier,highway,address,place,man_made,other_tags,poi_prompt
0,001Laaog8zdwYugH_HWEgw,21752,13.059461,21750.0,19.824303,21751.0,21.686056,44108.0,24.086535,21753.0,...,44109.0,44108.0,Colônia de Pescadores Z13,NaN,NaN,NaN,NaN,NaN,NaN,"Nearby,name is Colônia de Pescadores Z13,"
1,001Q3_sTWsNcUd0yl-nJLA,94058,17.821758,42113.0,21.367698,95019.0,21.512402,42137.0,21.694559,90529.0,...,95018.0,95019.0,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals""=>""signal""","Nearby,highway is traffic_signals,traffic_signals is signal,"
2,0025pq4iX5JFSzLE4mzwKQ,259,98.612218,48095.0,116.037520,52036.0,142.219760,48096.0,148.991137,52502.0,...,259.0,259.0,NaN,NaN,speed_camera,NaN,NaN,NaN,"""maxspeed""=>""70""","Nearby,highway is speed_camera,maxspeed is 70,"
3,002QPn5NUqaQIqgAXwrVlg,95628,125.756467,66004.0,167.137533,82889.0,177.383359,7174.0,196.736422,95632.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,002U0X8H3fkVK0dCi3ZRQA,93806,26.203495,93404.0,32.968048,17517.0,36.808813,22492.0,36.974101,93403.0,...,17517.0,17517.0,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,"Nearby,highway is traffic_signals,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161147,ZzZ0-G17tbu8HO4rUrh4-A,2414,42.792871,71755.0,90.276320,71756.0,163.123758,66588.0,171.256074,66287.0,...,2416.0,2414.0,NaN,NaN,NaN,NaN,NaN,NaN,"""bus""=>""yes"",""public_transport""=>""stop_position""","Nearby,bus is yes,public_transport is stop_position,"
161148,ZzZ9cMtim4UIwvi1I8vOAA,90160,51.994656,90161.0,70.898454,85677.0,102.030496,74035.0,136.287096,49739.0,...,90159.0,90160.0,NaN,NaN,crossing,NaN,NaN,NaN,"""crossing""=>""traffic_signals""","Nearby,highway is crossing,crossing is traffic_signals,"
161149,_zzZDJ5eTaqBpSWRrJlqmA,59005,6.677372,94904.0,6.997275,59012.0,9.174672,59006.0,10.105857,59011.0,...,94903.0,94904.0,NaN,NaN,street_lamp,NaN,NaN,NaN,NaN,"Nearby,highway is street_lamp,"
161150,zZznk-22KYhH72kTEaqLjw,41881,36.733770,9531.0,46.433957,9549.0,58.756743,9548.0,59.055031,9554.0,...,9530.0,9531.0,NaN,NaN,street_lamp,NaN,NaN,NaN,NaN,"Nearby,highway is street_lamp,"


In [67]:
# df1.to_excel(best_poi_path)

In [32]:

poi_df_nec.columns

Index(['Unnamed: 0', 'id', 'name', 'barrier', 'highway', 'address', 'place',
       'man_made', 'other_tags', 'poi_prompt'],
      dtype='object')

In [33]:
# desc_pd: file,prompt insert poi_prompt
# df1 : image_key, bestpoi
# poi_df_nec: FID ,poi_prompt
def connect(x):
    image_file_name = x[0]
    image_name =image_file_name.split('.')[0]
    best_poi_row = df1.loc[df1['img_key']==image_name]['best_poi']
    best_poi = None
    if best_poi_row.notna().any():
        best_poi=int(best_poi_row.iloc[0])
        poi_prompt = poi_df_nec.loc[poi_df_nec['id']==best_poi]['poi_prompt'].iloc[0]
        return poi_prompt
        # print(poi_prompt)
    # print(best_poi)
    # print("\n")
    

    # print(image_name)

In [34]:
result2 = desc_pd.apply(lambda x: connect(x), axis=1,result_type='expand')
result2

In [35]:
desc_pd['poi_prompt'] = result2

In [36]:
desc_pd

,key,prompt,poi_prompt
0,--29Da_btiAkuYBTI21ddw,"arafed building with a car parked in front of it, a matte painting inspired by Edi Rama, behance, hurufiyya, rosalia vila i tobella, found on google street view, location ( favela )",None
1,--8lKh4goK-MYUCkSM3eQw,"arafed building with a blue tarp on the ground next to a tree, a detailed matte painting by Thota Vaikuntham, cg society, samikshavad, bangalore, house on background, the neat and dense buildings",None
2,#NAME?,"arafed tree in a park with a fence and a street sign, a detailed matte painting by Narayan Shridhar Bendre, behance, bengal school of art, google point of view, old tree, streetview",None
3,#NAME?,"arafed building with a tree in front of it and a woman standing in front of it, a detailed matte painting by Thota Vaikuntham, behance, samikshavad, collapsed building, dilapidated houses, photo of poor condition",None
4,-0-2YRdnYF7a8LLSSOeNug,"there are people standing outside of a building with a sign on it, a detailed matte painting by Bholekar Srihari, cg society, samikshavad, lots of shops, on an indian street, business surrounding",None
...,...,...,...
101990,WOvdhoeEwmI43p4phwOdqA,"arafed view of a street at night with cars parked on the side of the road, a screenshot inspired by Gregory Crewdson, featured on flickr, samikshavad, city street view background, night camera, streetview","Nearby,name is Dr. Santosh Clinic,addr:district is Bengaluru Urban,addr:full is Tavarekere, Aicobo Nagar, BTM Layout, Bengaluru,addr:state is Karnataka,amenity is clinic,"
101991,M5zU1S4ziqFEeyfZto8E-g,"cars driving down a street with people walking and trees, a detailed matte painting by Thota Vaikuntham, featured on pexels, samikshavad, khajuraho, unobstructed road, city street view background","Nearby,name is Ashraya Badavane,highway is bus_stop,local_ref is Gollarahatti,"
101992,kAJDXwPyDknFCAS6c-Vxiw,"nighttime scene of a busy street with cars and motorcycles on it, a photo by Thota Vaikuntham, featured on flickr, samikshavad, in city street at night, bangalore, night time photograph","Nearby,name is Ericsson India Global Services,addr:city is Bangalore,addr:housename is Bagmane World Technology Center,addr:street is Outer Ring Road,office is company,"
101993,4ZgNCOzHa2tQorK-VbYNtg,"there are many trucks driving down the street under a bridge, a matte painting by Thota Vaikuntham, featured on pexels, samikshavad, bangalore, dhaka traffic, advanced highway","Nearby,name is Bismillah Dum Biryani,amenity is restaurant,"


In [37]:
desc_pd.to_excel(desc_poi_path)